### Enexis Kleinverbruikgegevens dataset
https://www.enexis.nl/over-ons/wat-bieden-we/andere-diensten/open-data 


Beschrijving:

https://www.enexis.nl/over-ons/-/media/documenten/open-data/toelichting-kleinverbruiksgegevens-open-data-enexis-netbeheer.pdf?modified=00010101000000&la=nl-nl&hash=1A66620D8DBA0D55A55767B507C535BC218B0B14

We hadden al de csv files per jaar vanaf 2010 maar er is nu ook een nieuwe csv file voor 2021 beschikbaar. op de website van Enexis staan alleen de laatste 3 jaar.

Belangrijkste attributen in de dataset:
- POSTCODE_VAN en POSTCODE_TOT, PC6
- PRODUCTSOORT, zowel Gas als Elektriciteit. Filter toepassen op alleen Elektriciteit
- AANSLUITINGEN_AANTAL, aantal aansluitingen in het postcode gebied
- LEVERINGSRICHTING_PERC, Percentage van de aansluitingen dat netto elektriciteits- of gasverbruik heeft. Dit percentage wordt lager naarmate er meer teruglevering plaatsvindt (b.v. vanwege zonnepanelen) 
- SOORT_AANSLUITING  de meest voorkomende ‘Soort aansluiting’, Opties voor elektriciteit: (”#zekeringen x # ampère”) 1x25, 1x35, 1x50, 3x25, 3x35, 3x50, 3x63, 3x80
- SJV_GEMIDDELD, Het gemiddeld standaardjaarverbruik (SJV) in kWh. Het verwachte jaarverbruik van een afnemer op een netaansluiting bij gestandaardiseerde condities en op basis van een genormaliseerd jaar. Bij teruglevering dan is het SJV gesaldeerd.

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import seaborn as sns

In [2]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

In [3]:
os.getcwd()

'C:\\Users\\MarcelKoolwijk\\Documents\\SQL\\jads_enexis\\notebooks'

In [4]:
# variables used in script
# path to Data folder
DATA = '../data/raw/Enexis_kleinverbruikgegevens'

# Ga naar de folder waarin de data staat
if 'Enexis_kleinverbruikgegevens' not in os.getcwd():
    os.chdir(DATA)

In [5]:
df = pd.DataFrame()
for j in glob.glob('Enexis_kleinverbruiksgegevens*.zip'):
    print(j)
    appenddata = pd.read_csv(j, sep=';', thousands='.', decimal=',',  encoding= 'unicode_escape')
    # jaar kolom toevoegen op basis van de file name
    appenddata['JAAR'] = j[-8:-4] # Het standaardjaarverbruik is het verwachte jaarverbruik gebaseerd op vorige jaar
    df = df.append(appenddata,ignore_index=True)

Enexis_kleinverbruiksgegevens_01012010.zip
Enexis_kleinverbruiksgegevens_01012011.zip
Enexis_kleinverbruiksgegevens_01012012.zip
Enexis_kleinverbruiksgegevens_01012013.zip
Enexis_kleinverbruiksgegevens_01012014.zip
Enexis_kleinverbruiksgegevens_01012015.zip
Enexis_kleinverbruiksgegevens_01012016.zip
Enexis_kleinverbruiksgegevens_01012017.zip
Enexis_kleinverbruiksgegevens_01012018.zip
Enexis_kleinverbruiksgegevens_01012019.zip
Enexis_kleinverbruiksgegevens_01012020.zip
Enexis_kleinverbruiksgegevens_01012021.zip


In [6]:
df.columns

Index(['NETBEHEERDER', 'NETGEBIED', 'STRAATNAAM', 'POSTCODE_VAN',
       'POSTCODE_TOT', 'WOONPLAATS', 'LANDCODE', 'PRODUCTSOORT',
       'VERBRUIKSSEGMENT', 'LEVERINGSRICHTING_PERC', 'AANSLUITINGEN_AANTAL',
       'FYSIEKE_STATUS_PERC', 'SOORT_AANSLUITING_PERC', 'SOORT_AANSLUITING',
       'SJV_GEMIDDELD', 'SJV_LAAG_TARIEF_PERC', 'SLIMME_METER_PERC', 'JAAR'],
      dtype='object')

In [7]:
# filter op elektriciteit
df = df[df['PRODUCTSOORT'] == 'ELK']

In [8]:
# check missing values
missing_total = df.isna().sum().sort_values(ascending=False)
print('Number of attributes with Null values: ' + str(len(missing_total[missing_total > 0])))
print(missing_total[missing_total > 0])

Number of attributes with Null values: 3
SOORT_AANSLUITING         107581
SOORT_AANSLUITING_PERC    107512
LEVERINGSRICHTING_PERC       958
dtype: int64


In [13]:
# fill missing values 
df['SOORT_AANSLUITING'] = df['SOORT_AANSLUITING'].fillna('Onbekend')
df['SOORT_AANSLUITING_PERC'] = df['SOORT_AANSLUITING_PERC'].fillna(0)
df['LEVERINGSRICHTING_PERC'] = df['LEVERINGSRICHTING_PERC'].fillna(0)

In [14]:
df.head()

,NETBEHEERDER,NETGEBIED,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,WOONPLAATS,LANDCODE,PRODUCTSOORT,VERBRUIKSSEGMENT,LEVERINGSRICHTING_PERC,AANSLUITINGEN_AANTAL,FYSIEKE_STATUS_PERC,SOORT_AANSLUITING_PERC,SOORT_AANSLUITING,SJV_GEMIDDELD,SJV_LAAG_TARIEF_PERC,SLIMME_METER_PERC,JAAR,PC4
0,Enexis B.V.,ENEXIS,Sasdijk,4251AB,4251AB,WERKENDAM,NL,ELK,KVB,100.0,16,100,0.0,Onbekend,4282.0,25.0,0.0,2010,4251
1,Enexis B.V.,ENEXIS,Sasdijk,4251AC,4251AC,WERKENDAM,NL,ELK,KVB,100.0,11,100,0.0,Onbekend,5113.0,10.0,0.0,2010,4251
2,Enexis B.V.,ENEXIS,Sasdijk,4251AD,4251AD,WERKENDAM,NL,ELK,KVB,100.0,30,100,0.0,Onbekend,4809.0,34.0,0.0,2010,4251
3,Enexis B.V.,ENEXIS,Nieuweweg,4251AE,4251AG,WERKENDAM,NL,ELK,KVB,100.0,21,100,0.0,Onbekend,5015.0,44.0,0.0,2010,4251
4,Enexis B.V.,ENEXIS,Koppenhof,4251AH,4251AH,WERKENDAM,NL,ELK,KVB,100.0,12,100,0.0,Onbekend,3074.0,22.0,0.0,2010,4251


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1391527 entries, 0 to 2563199
Data columns (total 19 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   NETBEHEERDER            1391527 non-null  object 
 1   NETGEBIED               1391527 non-null  object 
 2   STRAATNAAM              1391527 non-null  object 
 3   POSTCODE_VAN            1391527 non-null  object 
 4   POSTCODE_TOT            1391527 non-null  object 
 5   WOONPLAATS              1391527 non-null  object 
 6   LANDCODE                1391527 non-null  object 
 7   PRODUCTSOORT            1391527 non-null  object 
 8   VERBRUIKSSEGMENT        1391527 non-null  object 
 9   LEVERINGSRICHTING_PERC  1391527 non-null  float64
 10  AANSLUITINGEN_AANTAL    1391527 non-null  int64  
 11  FYSIEKE_STATUS_PERC     1391527 non-null  int64  
 12  SOORT_AANSLUITING_PERC  1391527 non-null  float64
 13  SOORT_AANSLUITING       1391527 non-null  object 
 14  SJ

In [16]:
# JAAR is nog object, omzetten naar float
df['JAAR'] = df['JAAR'].astype(float)

In [18]:
# voeg attibuut toe met numerieke deel van de postcode
df['PC4'] = df['POSTCODE_VAN'].astype('str').str[0:4].astype('int64')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1391527 entries, 0 to 2563199
Data columns (total 19 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   NETBEHEERDER            1391527 non-null  object 
 1   NETGEBIED               1391527 non-null  object 
 2   STRAATNAAM              1391527 non-null  object 
 3   POSTCODE_VAN            1391527 non-null  object 
 4   POSTCODE_TOT            1391527 non-null  object 
 5   WOONPLAATS              1391527 non-null  object 
 6   LANDCODE                1391527 non-null  object 
 7   PRODUCTSOORT            1391527 non-null  object 
 8   VERBRUIKSSEGMENT        1391527 non-null  object 
 9   LEVERINGSRICHTING_PERC  1391527 non-null  float64
 10  AANSLUITINGEN_AANTAL    1391527 non-null  int64  
 11  FYSIEKE_STATUS_PERC     1391527 non-null  int64  
 12  SOORT_AANSLUITING_PERC  1391527 non-null  float64
 13  SOORT_AANSLUITING       1391527 non-null  object 
 14  SJ

In [21]:
# extra attribuut toevoegen voor totale standaard jaarverbruik
df['SJV_TOTAAL'] = round(df['AANSLUITINGEN_AANTAL'] * df['SJV_GEMIDDELD'] / 1000) # delen door 1000 om MW ipv KW te krijgen

# Dataframe opslaan

In [22]:
os.getcwd()

'C:\\Users\\MarcelKoolwijk\\Documents\\SQL\\jads_enexis\\data\\raw\\Enexis_kleinverbruikgegevens'

In [23]:
# data locatie om de ht5 file op te slaan
data_processed_location = '../../processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [24]:
try:
    os.remove('kleinverbruikgegevens_data.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('kleinverbruikgegevens_data.h5')
store['kleinverbruikgegevens_data'] = df
store.close()

# Group by PC4

In [24]:
# group by pc4
df_pc4 = df.groupby(['PC4','JAAR']
                    , as_index=False
                    ).agg({'SJV_TOTAAL': 'sum'
                           ,'AANSLUITINGEN_AANTAL': 'sum'
                           ,'LEVERINGSRICHTING_PERC': 'mean'
                           }
                         )

In [25]:
df_pc4.head()

,PC4,JAAR,SJV_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC
0,4251,2010.0,21882.0,4336,99.953917
1,4251,2011.0,23084.0,4413,99.953917
2,4251,2012.0,24006.0,4486,99.854545
3,4251,2013.0,23728.0,4512,99.660633
4,4251,2014.0,23623.0,4544,99.094595
